In [5]:
import os
import json
import numpy as np
import pandas as pd

# Global stack for collecting keys and values
stack = []
datatype_stack = []

def accessJSON(data, parent_key=''):
    global stack, datatype_stack
    for key, value in data.items():
        full_key = f"{parent_key}.{key}" if parent_key else key
        
        if isinstance(value, dict):
            if 'datatype' in value:
                datatype_stack.append(f"{full_key}.datatype")
                datatype_stack.append(value['datatype'])
                value = value.get('value', '')  # Use 'value' or empty string if 'value' key does not exist
            stack.append(full_key)
            stack.append(value)
            if 'datatype' in value:
                stack.append(full_key)
                stack.append(value['value'])
        else:
            stack.append(full_key)
            stack.append(value)

def findingrowscolumns(keys):
    if not keys:
        return 0, 0
    
    start = ''
    ei = 0
    for i in range(len(keys)):
        if start == '':
            start = keys[i]
        elif keys[i] == start:
            ei = i
            break 
        else:
            ei = i 

    if ei == 0:
        return 0, 0

    count = len(keys) // ei
    return count, ei

def comparekeys(w1, w2):
    return w1 == w2

def seperatekeyvalues(stackelements):
    keys = []
    values = []
    for i in range(len(stackelements)):
        if i % 2 == 0:
            keys.append(stackelements[i])
        else:
            values.append(stackelements[i])
    return keys, values

def writingintoFile(fileName, rows, column, keys, values):
    try:
        a = np.array(values).reshape(rows, column)
    except ValueError:
        print(f"Error reshaping array of size {len(values)} into shape ({rows},{column}).")
        print(f"Values: {values}")
        return
    
    # Create DataFrame and write to CSV
    DF = pd.DataFrame(a, columns=keys[:column])
    DF.to_csv(fileName, index=False)

# Example JSON input
data = {
    "persons": [
        {
            "firstname": {
                "value": "John",
                "datatype": "xsd:string"
            },
            "lastname": "Doe",
            "lang": "en",
            "num": {
                "value": 3,
                "datatype": "xsd:integer"
            }
        },
        {
            "firstname": {
                "value": "Jane",
                "datatype": "xsd:string"
            },
            "lastname": "Smith",
            "lang": "fr",
            "num": {
                "value": "3.14",
                "datatype": "xsd:decimal"
            }
        }
    ]
}

# Call the function to extract keys and values
accessJSON(data)

# Separate keys and values
keys, values = seperatekeyvalues(stack)

# Determine the number of rows and columns
rows, columns = findingrowscolumns(keys)

# Include datatype columns
keys.extend(datatype_stack)
values.extend(datatype_stack)

# Debug print to check the computed rows and columns
print(f"Rows: {rows}, Columns: {columns}")
print(f"Keys: {keys}")
print(f"Values: {values}")

# Define the output CSV file name
outputFileName = 'output.csv'

# Write the data to the CSV file
writingintoFile(outputFileName, rows, columns, keys, values)

print(f"Data has been successfully written to {outputFileName}")

Rows: 0, Columns: 0
Keys: ['persons']
Values: [[{'firstname': {'value': 'John', 'datatype': 'xsd:string'}, 'lastname': 'Doe', 'lang': 'en', 'num': {'value': 3, 'datatype': 'xsd:integer'}}, {'firstname': {'value': 'Jane', 'datatype': 'xsd:string'}, 'lastname': 'Smith', 'lang': 'fr', 'num': {'value': '3.14', 'datatype': 'xsd:decimal'}}]]
Error reshaping array of size 1 into shape (0,0).
Values: [[{'firstname': {'value': 'John', 'datatype': 'xsd:string'}, 'lastname': 'Doe', 'lang': 'en', 'num': {'value': 3, 'datatype': 'xsd:integer'}}, {'firstname': {'value': 'Jane', 'datatype': 'xsd:string'}, 'lastname': 'Smith', 'lang': 'fr', 'num': {'value': '3.14', 'datatype': 'xsd:decimal'}}]]
Data has been successfully written to output.csv
